In [15]:
import pandas as pd
from MMRcall import MMRapi
from Cnx import Cnx
from fuzzywuzzy import fuzz
import make as mk
import difflib

"""
Platterfunc takes in a batch of VIN numbers and serves up
a piping hot set of residual values.
"""

# STATIC VARS
# ...

def main(vin):
    """
    Input: VIN.
    Output: Dict of residuals with axes: 
    {Make, Model, Trim, Term_1,..., Term_n, MileageBand_1,..., MileageBand_n}
    """
    conn = Cnx()
    mmr = MMRapi(vin)
    mmr.match()
    make = mmr.make()
    bod = mmr.body()
    model = mmr.model()
    year = mmr.model_year()
    """Access MMR Api"""
    slice_bymake = f"""
    SELECT [ALGResidualNewID] ,[ModelYear] ,[MakeNumber] ,[ModelNumber] ,[Style] ,[ModelDesc] ,[Description] ,[Month24] ,[Month30] ,[Month36] ,[Month42] ,[Month48] ,[Month54] ,[Month60] FROM [vinput].[dbo].[ALGResidualNewTable20220708]
    WHERE MakeNumber = {mk.make_number[make]} AND
    ModelYear = {year}
    """
    conn.set_query(slice_bymake)
    make_match = conn.execute()
    df = pd.DataFrame(make_match)
    df.columns = [ 'ALGResidualNewID', 'ModelYear', 'MakeNumber', 'ModelNumber', 'Style', 'ModelDesc', 'Description', 'Month24', 'Month30', 'Month36', 'Month42', 'Month48', 'Month54', 'Month60' ]

    best_fit = difflib.SequenceMatcher()
    best_fit.set_seq1(bod)
    def find_it(x):
        best_fit.set_seq2(x)
        return best_fit.ratio()
    
    df['Fuzzy'] = df[:]['Description'].map(lambda x: find_it(x))
    scope = df.loc[df.Fuzzy == df.Fuzzy.max()]
    


    
    
main('5UXCR6C09N9M97942')

In [16]:
s = difflib.SequenceMatcher(None, " im just affpples to appffffle", "apples to apples").ratio()
s

0.6521739130434783

In [24]:
import pandas as pd
from MMRcall import MMRapi
from Cnx import Cnx
from fuzzywuzzy import fuzz
import make as mk
import difflib

"""
Platterfunc takes in a batch of VIN numbers and serves up
a piping hot set of residual values.
"""

# STATIC VARS
# ...

def main(vin):
    """
    Input: VIN.
    Output: Dict of residuals with axes: 
    {Make, Model, Trim, Term_1,..., Term_n, MileageBand_1,..., MileageBand_n}
    """
    conn = Cnx()
    mmr = MMRapi(vin)
    mmr.match()
    make = mmr.make()
    bod = mmr.body()
    model = mmr.model()
    year = mmr.model_year()
    """Access MMR Api"""

    if make == 'Ford' or make == 'Chevrolet' or make == 'Dodge':
        slice_bymake = f"""
            SELECT [ALGResidualNewID] ,[ModelYear] ,[MakeNumber] ,[ModelNumber] ,[Style] ,[ModelDesc] ,[Description], [Region] ,[Month24] ,[Month30] ,[Month36] ,[Month42] ,[Month48] ,[Month54] ,[Month60] FROM [vinput].[dbo].[ALGResidualNewTable20220708]
            WHERE MakeNumber = {mk.make_number[make][0]} or MakeNumber = {mk.make_number[make][1]}  AND
            ModelYear = {year}
        """
    else:
        slice_bymake = f"""
            SELECT [ALGResidualNewID] ,[ModelYear] ,[MakeNumber] ,[ModelNumber] ,[Style] ,[ModelDesc] ,[Description], [Region] ,[Month24] ,[Month30] ,[Month36] ,[Month42] ,[Month48] ,[Month54] ,[Month60] FROM [vinput].[dbo].[ALGResidualNewTable20220708]
            WHERE MakeNumber = {mk.make_number[make]} AND
            ModelYear = {year}
        """
    
    conn.set_query(slice_bymake)
    make_match = conn.execute()
    df = pd.DataFrame(make_match)
    df.columns = [ 'ALGResidualNewID', 'ModelYear', 'MakeNumber', 'ModelNumber', 'Style', 'ModelDesc', 'Description', 'Region', 'Month24', 'Month30', 'Month36', 'Month42', 'Month48', 'Month54', 'Month60' ]

    def find_it(x):
        return fuzz.partial_ratio(bod, x)


    best_fit = difflib.SequenceMatcher()
    best_fit.set_seq1(bod)
    def find_it_fast(x):
        best_fit.set_seq2(x)
        return best_fit.real_quick_ratio()
    
    df['Fuzzy'] = df[:]['Description'].map(lambda x: find_it_fast(x))
    
    df = df[df.ModelDesc == model]
    df = df[df.Fuzzy == df.Fuzzy.max()]
    return df

    


    
    
main('5UXCR6C09N9M97942')

,ALGResidualNewID,ModelYear,MakeNumber,ModelNumber,Style,ModelDesc,Description,Region,Month24,Month30,Month36,Month42,Month48,Month54,Month60,Fuzzy
64,8085851,2022,100,180,110,X5,4dr Wgn sDrive40i,N,33,31,29,25,22,14,10,0.607143
65,8085852,2022,100,180,115,X5,4dr Wgn xDrive40i,N,35,33,31,27,24,16,10,0.607143
66,8085853,2022,100,180,132,X5,4dr Wgn xDrive45e,N,36,34,32,28,25,17,10,0.607143
74,8091737,2022,100,180,110,X5,4dr Wgn sDrive40i,A,33,31,29,25,22,14,10,0.607143
75,8091738,2022,100,180,115,X5,4dr Wgn xDrive40i,A,35,33,31,27,24,16,10,0.607143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12632,8789183,2022,100,180,132,X5,4dr Wgn xDrive45e,C,36,34,32,27,24,16,10,0.607143
12641,9026116,2022,100,180,132,X5,4dr Wgn xDrive45e,N,36,34,32,26,23,16,10,0.607143
12678,9045430,2022,100,180,110,X5,4dr Wgn sDrive40i,A,32,30,28,22,19,12,10,0.607143
12679,9045431,2022,100,180,115,X5,4dr Wgn xDrive40i,A,34,32,30,24,21,14,10,0.607143


In [24]:
import requests
import pandas as pd
from Cnx import Cnx

def get_styleID(vin):
    url = f'https://apitesting.vinwhiz.com/api/v1.1/vins/{vin}/ris'
    api_key = '2D979E71-EE01-4B7D-A995-F2A2D29A5E85'
    r = requests.get(url, headers={"x-apikey": f"{api_key}", "accept": "application/json"})
    #print(r.json()['signals'][0]['styleId'])
    return r.json()['signals'][0]['styleId']

def vin_year(vin):
    url = f'https://apitesting.vinwhiz.com/api/v1.1/vins/{vin}/ris'
    api_key = '2D979E71-EE01-4B7D-A995-F2A2D29A5E85'
    r = requests.get(url, headers={"x-apikey": f"{api_key}", "accept": "application/json"})
    return r.json()['signals'][0]['vehicleHints']['YEAR']

# print(get_styleID('5UXCR6C09N9M97942'))

def style_to_ALGcode(styleId):
    chrmap = pd.read_csv('ALG_US_CHROMEMAP.csv')
    chrmap = chrmap[chrmap.ChromeStyleId == styleId]
    return chrmap['AlgCode'].iloc[0] 

def vin_alg(vin):
    return style_to_ALGcode(get_styleID(vin))

def slice(i, end, start=0):
    return str(i)[start:end]

def main(vin):
    """
    Main functional interface for VINPUT.
    Input: VIN Number.
    Output: Dataframe
    """
    algcode = vin_alg(vin)
    make_number = slice(algcode, 3)
    model_num = slice(algcode, 6, 3)
    style_num = slice(algcode, 9, 6)
    model_year = vin_year(vin)
    conn = Cnx()
    quer = f"""
        SELECT
        [ALGResidualNewID]
        [ModelYear],
        [ModelDesc],
        [Description], 
        [MakeNumber],
        [ModelNumber],
        [Style],
        [Region],
        [EffectiveDate]
        FROM [vinput].[dbo].[ALGResidualNewTable20220708]
        WHERE [MakeNumber] = {make_number} AND
        [ModelNumber] = {model_num} AND
        [Style] = {style_num} AND
        [ModelYear] = {model_year} AND
        [Region] = 'N'
    """
    conn.set_query(quer)
    make_match = conn.execute()
    df = pd.DataFrame(make_match)
    df = df[df[7] == df[7].max()]
    return df

main('5UXCR6C09N9M97942')

,0,1,2,3,4,5,6,7
25,9038321,X5,4dr Wgn xDrive40i,100,180,115,N,2022-07-01 00:00:00.0000000


In [ ]:
from Cnx import Cnx

conn = Cnx()
quer = f"""
        SELECT 
        [ModelYear],
        [Description], 
        [MakeNumber],
        [ModelNumber],
        [Style] 
        FROM [vinput].[dbo].[ALGResidualNewTable20220708]
        WHERE {make_number} + [ModelNumber] + [Style] = {algcode}
    """
conn.set_query(quer)

In [34]:
i = 123456789
j = str(i)[:3]
k = str(i)[3:6]
l = str(i)[6:9]
int(l)

789